# Lab 3.2 – Understanding Detection Metrics

In this lab, we manually compute IoU (Intersection over Union), implement an IoU function in Python, test multiple scenarios, and analyze YOLOv8 validation metrics including mAP@0.5 and mAP@0.5:0.95.

Objective:
- Understand how IoU measures overlap between bounding boxes.
- Implement IoU in Python.
- Evaluate object detection performance using YOLOv8.
- Interpret mAP metrics for real-world deployment.

## Manual IoU Calculation

Given:

Ground Truth Box (GT): (2, 2, 6, 6)  
Prediction Box (Pred): (4, 4, 8, 8)

Step 1: Intersection Coordinates
x_left = max(2, 4) = 4  
y_top = max(2, 4) = 4  
x_right = min(6, 8) = 6  
y_bottom = min(6, 8) = 6  

Intersection width = 6 − 4 = 2  
Intersection height = 6 − 4 = 2  
Intersection Area = 2 × 2 = 4  

Step 2: Individual Areas  
Area(GT) = 4 × 4 = 16  
Area(Pred) = 4 × 4 = 16  

Step 3: Union  
Union = 16 + 16 − 4 = 28  

IoU = 4 / 28 = 0.1428  

Final IoU ≈ 0.14

In [1]:
def calculate_iou(boxA, boxB):
    """
    Calculate Intersection over Union (IoU)
    box format: (x1, y1, x2, y2)
    """
    x_left = max(boxA[0], boxB[0])
    y_top = max(boxA[1], boxB[1])
    x_right = min(boxA[2], boxB[2])
    y_bottom = min(boxA[3], boxB[3])

    # No overlap case
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    union_area = areaA + areaB - intersection_area

    return intersection_area / union_area

In [2]:
# 1. Perfect match (IoU = 1.0)
box1 = (2, 2, 6, 6)
box2 = (2, 2, 6, 6)
print("Perfect Match IoU:", calculate_iou(box1, box2))

# 2. Partial overlap (~0.14 from manual calculation)
box3 = (2, 2, 6, 6)
box4 = (4, 4, 8, 8)
print("Partial Overlap IoU:", calculate_iou(box3, box4))

# 3. No overlap (IoU = 0.0)
box5 = (0, 0, 2, 2)
box6 = (3, 3, 5, 5)
print("No Overlap IoU:", calculate_iou(box5, box6))

Perfect Match IoU: 1.0
Partial Overlap IoU: 0.14285714285714285
No Overlap IoU: 0.0


In [3]:
from ultralytics import YOLO

# Load model
model = YOLO("yolov8n.pt")

# Run validation
metrics = model.val(data="coco128.yaml")

print(metrics)

100%|██████████| 6.23M/6.23M [00:01<00:00, 4.22MB/s]


Ultralytics YOLOv8.1.0 🚀 Python-3.10.10 torch-2.1.2+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

Dataset 'coco128.yaml' images not found ⚠️, missing path 'C:\Users\HP\Documents\datasets\coco128\images\train2017'


100%|██████████| 6.66M/6.66M [00:03<00:00, 2.12MB/s]
Unzipping C:\Users\HP\Documents\datasets\coco128.zip to C:\Users\HP\Documents\datasets\coco128...: 100%|██████████| 263/263 [00:00<00:00, 1263.11file/s]

Dataset download success ✅ (6.9s), saved to C:\Users\HP\Documents\datasets



100%|██████████| 755k/755k [00:00<00:00, 2.85MB/s]
val: Scanning C:\Users\HP\Documents\datasets\coco128\labels\train2017... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<00:00, 184.10it/s]

val: New cache created: C:\Users\HP\Documents\datasets\coco128\labels\train2017.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:13<00:00,  1.69s/it]


                   all        128        929       0.64      0.537      0.605      0.446
                person        128        254      0.797      0.677      0.764      0.538
               bicycle        128          6      0.514      0.333      0.315      0.264
                   car        128         46      0.813      0.217      0.273      0.168
            motorcycle        128          5      0.687      0.887      0.898      0.685
              airplane        128          6       0.82      0.833      0.927      0.675
                   bus        128          7      0.491      0.714      0.728      0.671
                 train        128          3      0.534      0.667      0.706      0.604
                 truck        128         12          1      0.332      0.473      0.297
                  boat        128          6      0.226      0.167      0.316      0.134
         traffic light        128         14      0.734        0.2      0.202      0.139
             stop sig

## Interpretation of YOLO Metrics

mAP@0.5 represents the mean Average Precision at an IoU threshold of 0.5. 
This means a predicted bounding box is considered correct if it overlaps 
with the ground truth by at least 50%.

A high mAP@0.5 indicates the model is good at detecting objects, but not 
necessarily precise in localization.

mAP@0.5:0.95 averages performance across multiple IoU thresholds 
(from 0.5 to 0.95). This is a stricter and more realistic metric 
because it evaluates how well the model performs under tighter 
localization requirements.

For deployment:
- If mAP@0.5 is high but mAP@0.5:0.95 is much lower, 
  the model detects objects but bounding boxes are not very precise.
- For safety-critical systems (autonomous driving, surveillance),
  mAP@0.5:0.95 is more important.
- For general detection (retail counting, basic monitoring),
  mAP@0.5 may be sufficient.